In [ ]:
import dotenv from "dotenv";
dotenv.config({ path: '.env.development.local' });

import OpenAI from "openai";

// Deno imports
import { Stagehand } from "npm:@browserbasehq/stagehand";
import { z } from "npm:zod";
import { underscoreToCapitalCase, run, handleDropDown, initLocalOpenAI, askYesNoGPT, askGptForOption, multipleOptionGPT, initLocalStagehand} from "./utils.js";
import { lettuceCustomerDataStub } from "./data-stubs.js";
import { formationsData } from "./customerPayrollData.js";
import { generateStateAddress } from "./index.js";

const cdpPort = 9222;

const page = await initLocalStagehand();

const GPT = new OpenAI({
    apiKey: process.env.OPENAI_API_KEY,
});

let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
let NAICS = 812310;

In [ ]:

//Santa Clara???
let data = lettuceCustomerDataStub
data.businessAddress = await generateStateAddress('Florida')
data.mailingAddress = await generateStateAddress('Florida')

console.log(data.businessAddress)
console.log(data.mailingAddress)


In [ ]:

console.log(data)

In [ ]:
const step = async () => {
  await page.goto('https://taxapps.floridarevenue.com/taxregistration');
  await page.waitForTimeout(3_000);

  await page.act("Click on the 'Create User Profile' button");

}

run(step);

In [ ]:

const step = async () => {
    let user = "skunkworksautoFL2";
    let email = "skunkworksauto0+FL2@gmail.com";
    let pass = "F0rm@tions";

    let username_Boxes = await page.observe("Find the 'Required Username' texbox and the 'Confirm Username' textbox");
    let email_Boxes = await page.observe("Find the 'Required Email Address' texbox and the 'Confirm Email Addres' textbox");
    let password_Boxes = await page.observe("Find the 'Required Password' texbox and the 'Confirm Password' textbox");

    username_Boxes[0].arguments = [user];
    username_Boxes[1].arguments = [user];

    email_Boxes[0].arguments = [email];
    email_Boxes[1].arguments = [email];

    password_Boxes[0].arguments = [pass];
    password_Boxes[1].arguments = [pass];
    
    await page.act(username_Boxes[0]);
    await page.act(username_Boxes[1]);

    await page.act(email_Boxes[0]);
    await page.act(email_Boxes[1]);

    await page.act(password_Boxes[0]);
    await page.act(password_Boxes[1]);

    await page.act("Click on the 'Create New User Profile' Button");
}

run(step);

In [ ]:

const verify = async () => {
    let email = "skunkworksauto0@gmail.com"
    let password = "Lettuce99!"

    await page.goto('https://accounts.google.com');
    await page.waitForTimeout(3_000);
    
    await page.act(`Fill out the "Email or phone" textbox with ${email}`);
    await page.act("Click on the 'Next' Button");
    await page.waitForTimeout(3_000);
    
    await page.act(`Fill out the "Enter your password" textbox with ${password}`);
    await page.act("Click on the 'Next' Button");
    await page.waitForTimeout(3_000);

    await page.goto('https://mail.google.com/mail/u/0/#inbox');
    await page.waitForTimeout(3_000);
    
    await page.act("Under the PRIMARY section, click on the top-most 'Florida Department' link");
    await page.waitForTimeout(3_000);
    
    await page.act("Click on the extended URL link");
    await page.waitForTimeout(3_000);

    await page.act("Click on the 'Log in' link");
    await page.waitForTimeout(3_000);    
}

run(verify);

In [ ]:

const Login = async () => {
    let user = "skunkworksautoFL2";
    let pass = "F0rm@tions";

    await page.act(`Fill out the "Username" textbox with ${user}`);
    await page.act(`Fill out the "Enter your password" textbox with ${pass}`);

    await page.act("Click on the 'Login' Button");
}

run(Login);

In [ ]:

const Step = async () => {

    await page.act("Click on the 'Start New Application' Button");
    
}

run(Step);

In [ ]:

const Step = async () => {

    await handleDropDown("Find the 'Reason for Applying' drop down", 'New' );
    await page.waitForTimeout(2_000);

    await page.act(`Fill out the "Date of first Florida taxable activity" textbox with ${data.dateFiled}`);
    await page.act("Click on the 'Save & Continue' button");
}

run(Step);

In [ ]:

const Step = async () => {

    await page.act(`Fill out the "Legal Name of Business" textbox with ${data.organizationLegalName}`);

    await page.act(`Fill out the "Telephone Number" textbox with ${data.businessPhoneNumber}`);

    await page.act("Click on the 'No' checkbox");

    await page.act("Click on the 'Save & Continue' button");
}

run(Step);

In [ ]:
const Step = async () => {
    
    let q = await page.extract("Find the first statement corresponding to Yes/No buttons on the page");
    let q2 = await page.extract("Find the second statement corresponding to Yes/No buttons on the page");
   
    let options = await page.observe(`Find the Yes/No radio buttons for the FIRST question: '${q}'`);
    let options2 = await page.observe(`Find the Yes/No radio buttons for the SECOND question: '${q2}'`);

    let answer = await askYesNoGPT(description, q, GPT);
    let answer2 = await askYesNoGPT(description, q2, GPT);

    await page.act(options[answer]);

    await page.act(options2[answer2]);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {

    await handleDropDown("Find the 'Florida County' drop down", data.businessAddress.county);
   
    let businessAddress_modal = await page.observe("In the 'Physical Location of Business or Rental Property' section, find the Street Address, City and Postal Code textboxes");
    let mailingAddress_modal = await page.observe("In the 'Business Mailing Address' section, find the Street Address, City and Postal Code textboxes");

    let bStreet = businessAddress_modal[0];
    let bCity = businessAddress_modal[1];
    let bPostal = businessAddress_modal[2];

    let mStreet = mailingAddress_modal[0];
    let mCity = mailingAddress_modal[1];
    let mPostal = mailingAddress_modal[2];

    bStreet.arguments = [data.businessAddress.street1];
    bCity.arguments = [data.businessAddress.city];
    bPostal.arguments = [data.businessAddress.postal + "1234"];

    mStreet.arguments = [data.businessAddress.street1]
    mCity.arguments = [data.businessAddress.city]
    mPostal.arguments = [data.businessAddress.postal + "1234"];

    await page.act(bStreet);
    await page.act(bCity);
    await page.act(bPostal);

    await page.act(mStreet);
    await page.act(mCity);
    await page.act(mPostal);

    await page.act("Click on the 'Save & Continue' button");
    await page.waitForTimeout(3_000);

    let confirm_modal = await page.observe("There are two 'I confirm the address entered is correct' checkboxes, find them.");

    await page.act(confirm_modal[0]);
    await page.act(confirm_modal[1]);

        //Differentiate better with just act
    /*
    await page.act("In the 'Physical Location of Business or Rental Property' section, click on the 'I confirm the address entered is correct' checkbox");
    await page.act("In the 'Business Mailing Address' section, click on the 'I confirm the address entered is correct' checkbox");
    */

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {

    await page.act("Click on the 'NAICS Code' radio button");

    await page.act("Click on the 'Save & Continue' button");



}

run(Step);

In [ ]:
const Step = async () => {
    await page.act("Click on the 'Enter Known NAICS Code' radio button");

    await page.act(`Fill out the "Enter NAICS Code" textbox with ${NAICS}`);

    await page.act("Click on the 'Search' button");

    // Use query instead of observe to get all current dropdown options
    let SIC_options_raw = await page.query("Get all options from the 'Select Your SIC Description' dropdown");

    // Map to extract just the descriptions (or whatever identifier GPT needs)
    let SIC_options = SIC_options_raw.map(opt => opt.description);

    // Ask GPT which is the best option
    let selectedIndex = await askGptForOption(description, "Select Your SIC Description", SIC_options, GPT);

    console.log(SIC_options_raw);
    console.log("Selected index:", selectedIndex);
    console.log("Chosen option:", SIC_options_raw[selectedIndex]);

    // Now act on the selected option
    await page.act(`Select '${SIC_options_raw[selectedIndex].description}' from the 'Select Your SIC Description' dropdown`);
    
    await handleDropDown("Select Your SIC Description", SIC_options[answer]);
    
}

run(Step);

In [ ]:
const Step = async () => {
  await page.act("Click on the 'Enter Known NAICS Code' radio button");

  await page.act(`Fill out the "Enter NAICS Code" textbox with ${NAICS}`);

  await page.act("Click on the 'Search' button");

  await page.act("Click on the 'Select Your SIC Description' dropdown");

  // Extract dropdown text
  const SIC_options_raw = await page.extract("Get all options from the 'Select Your SIC Description' dropdown");

  const SIC_options = SIC_options_raw.extraction
    .split("\n")
    .map(s => s.trim())
    .filter(s => s && !s.toLowerCase().includes("select"));

  const selectedIndex = await askGptForOption(description, "Select Your SIC Description", SIC_options, GPT);

  await handleDropDown("Select Your SIC Description", SIC_options[selectedIndex]);

  await page.act("Click on the 'Add' button");

  await page.act("Click on the 'Ok' button");

  await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {
  
  await handleDropDown("Select your form of business ownership", "Limited");

  await page.act("Click on the 'Save & Continue' button");

}
  
run(Step);

In [ ]:
const Step = async () => {
  
  // single or multi-member??

  await handleDropDown("Membership Type", "Single");

  await handleDropDown("Single-Member Type", "S");

  await page.act(`Fill out the "Federal Employer Identification Number (FEIN)" textbox with ${data.employerIdentificationNumber}`);

  await page.act("Click on the 'Save & Continue' button");

}
  
run(Step);

In [ ]:
const Step = async () => {
  
  await page.act(`Fill out the "First Name" textbox with ${data.firstName}`);
  await page.act(`Fill out the "Last Name" textbox with ${data.lastName}`);
  await page.act(`Fill out the "Title" textbox with ${"Member"}`);
  
  await page.act(`Fill out the "Identifier" textbox with ${data.socialSecurityNumber}`);

  await page.act(`Fill out the "Telephone Number" textbox with ${data.businessPhoneNumber}`);

  await handleDropDown("Existing Addresses on Record", data.businessAddress.street1);
  
  await page.act("Click on the 'Add Officer' button");

  await page.act("Click on the 'Ok' button");
  
  await page.act("Click on the 'Save & Continue' button");

}
    
run(Step);

In [ ]:
const Step = async () => {
  
    await page.act(`Fill out the "Date of Florida Incorporation or Organization" textbox with ${data.dateFiled}`);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
const Step = async () => {
    
    //Did we form it? Can it be known by another name in our case?

    await page.act("On the 'Has your business ever been known by another name?' field, click the 'No' Button");
    
    await page.act("On the 'Was that business issued a Florida certificate of registration or tax account number?' field, click the 'No' Button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Sales Rentals or Repairs of Products

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Property Rentals, Leases, or Licenses

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Real Property Contractors

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Services

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Fuels

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Secondhand Goods or Scrap Metal

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Coin-Operated Amusement Machines

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Coin-Operated Vending Machines

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Purchases

//Wierd edge-case [0, 1]

const Step = async () => {

    let q = await page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await page.observe(`Find every checkbox available for the instruction: '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);

    let answers = await multipleOptionGPT(description, q, optionDescriptions, GPT);

    console.log(answers);
    for (const idx of answers) {
        await page.act(options[idx]);
    }
    
    //await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Prepaid Wireless E911 Fee
const Step = async () => {

    let q = await page.extract("Find the statement corresponding to the Yes/No buttons on the page");
   
    let options = await page.observe(`Find the Yes/No radio buttons for the question: '${q}'`);

    let answer = await askYesNoGPT(description, q, GPT);

    await page.act(options[answer]);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Solid Waste
const Step = async () => {

    let q1 = await page.extract("Find the FIRST statement corresponding to the Yes/No buttons on the page");
    let q2 = await page.extract("Find the SECOND statement corresponding to the Yes/No buttons on the page");
    let q3 = await page.extract("Find the THIRD statement corresponding to the Yes/No buttons on the page");
   
    let options1 = await page.observe(`Find the Yes/No radio buttons for the FIRST question: '${q1}'`);
    let options2 = await page.observe(`Find the Yes/No radio buttons for the SECOND question: '${q2}'`);
    let options3 = await page.observe(`Find the Yes/No radio buttons for the THIRD question: '${q3}'`);

    let answer1 = await askYesNoGPT(description, q1, GPT);
    let answer2 = await askYesNoGPT(description, q2, GPT);
    let answer3 = await askYesNoGPT(description, q3, GPT);

    await page.act(options1[answer1]);
    await page.act(options2[answer2]);
    await page.act(options3[answer3]);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Gross Receipts Tax on Dry-Cleaning
const Step = async () => {

    let q = await page.extract("Find the statement corresponding to the Yes/No buttons on the page");
   
    let options = await page.observe(`Find the Yes/No radio buttons for the question: '${q}'`);

    let answer = await askYesNoGPT(description, q, GPT);

    await page.act(options[answer]);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Determination of Tax Liabilities
const Step = async () => {

    /*
    await page.act("On the 'Do you have or will you have, employees in Florida?' field, click on the 'Yes' button");
    await page.act("On the 'Do you, or will you, lease workers from an employee leasing company to work in Florida?' field, click on the 'No' button");
    await page.act("On the 'Do you use the services of persons in Florida whom you consider to be self-employed, independent contractors other than those engaged in a distinct business, occupation, or profession that serves the general public (e.g., plumber, general contractor, or certified public accountant)?' field, click on the 'No' button");
    */

    await page.act("On the FIRST question, click on the 'Yes' button");
    await page.act("On the SECOND question, click on the 'No' button");
    await page.act("On the THIRD question, click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

## Missing Additional Questions, Locations with Employees, Payroll Agent, Mailing Addresses (multiple questions), and Employer Details ##

In [ ]:
//Communications Services Tax
const Step = async () => {
    
    await page.act("Click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Direct Pay Permit
const Step = async () => {
    
    await page.act("Click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Documentary Stamp Tax
const Step = async () => {
    
    await page.act("Click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Gross Receipts Tax on Electrical Power and Gas
const Step = async () => {

    /*
    await page.act("On the 'Do you own or operate an electric or natural or manufactured gas (LP gas is excluded) utility distribution facility in Florida?' field, click on the 'Yes' button");
    await page.act("On the 'Do you import natural or manufactured gas (LP gas is excluded) into Florida for your own use?' field, click on the 'No' button");
    */
    
    await page.act("Click on the FIRST 'No' button");
    await page.act("Click on the SECOND 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Severance Taxes in Miami Dade 39 & 40??
const Step = async () => {
    
    await page.act("Click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Enrollment Introduction
const Step = async () => {
    
    await page.act("Click on the 'No' button");

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Authorization for Email Communication

//Same info as Formations Corp??
const Step = async () => {
    
    await page.act("Click on the 'I authorize' checkbox");

    await page.act(`Fill out the "First Name" textbox with ${"Formations"}`);
    await page.act(`Fill out the "Last Name" textbox with ${"Corp"}`);

    await page.act(`Fill out the "Telephone Number" textbox with ${"425-292-2008"}`);

    await page.act(`Fill out the "Email Address" textbox with ${"accounting@formationscorp.com"}`);

    await page.act("Click on the 'Save & Continue' button");

}

run(Step);

In [ ]:
//Step 43 Authorization for Email Communication

//Same info as Formations Corp??
const Step = async () => {
    
    await handleDropDown("In the 'Declaration' section, find the 'Available Authorities' combobox.", data.lastName);

    await page.act(`In the 'Declaration' section, fill out the "First Name" textbox with ${data.firstName}`);
    await page.act(`In the 'Declaration' section, fill out the "Last Name" textbox with ${data.lastName}`);

    await handleDropDown("In the 'Signature' section, find the 'Available Signatories' combobox.", data.lastName);

    await page.act(`In the 'Signature' section, fill out the "First Name" textbox with ${data.firstName}`);
    await page.act(`In the 'Signature' section, fill out the "Last Name" textbox with ${data.lastName}`);
    await page.act(`In the 'Signature' section, fill out the "Title" textbox with ${"Member"}`);

    await page.act("Click on the 'I certify' button");

    //click on Submit Application

    //click on Submit

    //Step 44 Print to PDF the Application confirmation into their Permanent Folder

    //Step 45 3-5 business days we should log back in to get the information; the customer will receive application confirmation in 7-10 via US mail

}

run(Step);